In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from folium.plugins import HeatMap

# Cargar los datos de las secciones censales con la renta
secciones_censales = gpd.read_file('secciones_censales_barcelona.geojson')
secciones_censales = secciones_censales[['codi_districte', 'codi_barri', 'codi_seccio_censal', 'Import_Euros', 'geometry']]

# Cargar los puntos de referencia
refpoints = gpd.read_file('hundredpoints.geojson')

# Establecer el mismo sistema de referencia de coordenadas (CRS) para ambos GeoDataFrames
secciones_censales.set_crs(epsg=4326, inplace=True)
refpoints.set_crs(epsg=4326, inplace=True)

# Convertir el CRS a uno adecuado para los cálculos de distancia
secciones_censales = secciones_censales.to_crs(epsg=3857)
refpoints = refpoints.to_crs(epsg=3857)

# Crear una lista para almacenar las rentas promedio y buffers
avg_rent_list = []

# Calcular la renta promedio dentro de un radio de 750 metros para cada punto
for idx, point in refpoints.iterrows():
    buffer_geom = point['geometry'].buffer(750)  # Buffer de 750 metros
    seccions_in_buffer = secciones_censales[secciones_censales.geometry.intersects(buffer_geom)]
    avg_rent = seccions_in_buffer['Import_Euros'].mean()
    avg_rent_list.append({'geometry': point['geometry'].centroid, 'avg_rent': round(avg_rent, 2), 'index': idx})

# Convertir avg_rent_list a GeoDataFrame
results_df = pd.DataFrame(avg_rent_list)
results_gdf = gpd.GeoDataFrame(results_df, geometry='geometry')

# Establecer el CRS para el GeoDataFrame de resultados
results_gdf.set_crs(epsg=3857, inplace=True)

# Reproyectar de nuevo a WGS 84 para visualización
results_gdf = results_gdf.to_crs(epsg=4326)

# Crear un mapa centrado en la media de la longitud y latitud de los puntos de referencia
map_center = [results_gdf.geometry.y.mean(), results_gdf.geometry.x.mean()]
m = folium.Map(location=map_center, zoom_start=12)

# Añadir los puntos y las rentas promedio al mapa
for idx, row in results_gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f'Avg Rent: {row["avg_rent"]}',
        tooltip=row["avg_rent"]
    ).add_to(m)

# Crear un mapa de calor ajustando los colores para visualizar mejor las diferencias
heat_data = [[row.geometry.y, row.geometry.x, row['avg_rent']] for idx, row in results_gdf.iterrows()]
heat_map = folium.Map(location=map_center, zoom_start=12)

# Ajuste de los colores del mapa de calor
HeatMap(heat_data, min_opacity=0.3, max_zoom=18, radius=15, blur=15, max_value=results_df['avg_rent'].max(),
        gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1.0: 'red'}).add_to(heat_map)

# Guardar los mapas en archivos HTML
map_path = 'Mapas/average_rent.html'
heatmap_path = 'Mapas/heat_map.html'
results_csv_path = 'Resultado/average_rent_results.csv'

m.save(map_path)
heat_map.save(heatmap_path)

# Guardar la tabla de resultados en un archivo CSV
results_df.to_csv(results_csv_path, index=False)

print('Mapas y tabla de resultados generados:')
print(map_path)
print(heatmap_path)
print(results_csv_path)


Mapas y tabla de resultados generados:
Mapas/average_rent.html
Mapas/heat_map.html
Resultado/average_rent_results.csv
